In [ ]:
# --- Session 1: Training & Saving ---

# Step 1: Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Install packages
!pip install mne tensorflow seaborn --quiet

# Step 3: Imports and setup
import os
import numpy as np
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
import mne
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Step 4: Constants and paths
BASE_DATA_PATH = '/content/drive/MyDrive/BILSTM_MODEL/edf_files/'
ANNOTATION_BASE_PATH = '/content/drive/MyDrive/BILSTM_MODEL/textfile/'

CHUNK_SIZE_SECONDS = 5
SAMPLING_RATE = 256

# Step 5: Helper functions (exact from your code)
def time_str_to_seconds(t_str):
    h, m, s = map(int, t_str.split('.'))
    return h * 3600 + m * 60 + s

def load_edf(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True, verbose=False)
    raw.resample(SAMPLING_RATE)
    return raw

def load_seizure_times(annotation_path):
    times = []
    reg_start = None
    with open(annotation_path, 'r') as f:
        for line in f:
            if 'Registration start time:' in line:
                reg_start = time_str_to_seconds(line.strip().split(': ')[1])
            elif 'Seizure start time:' in line and reg_start is not None:
                seizure_time = time_str_to_seconds(line.strip().split(': ')[1])
                times.append(seizure_time - reg_start)
    return times

def segment_eeg(raw, start, end):
    sfreq = int(raw.info['sfreq'])
    data = raw.get_data()
    chunks = []
    for t in range(start, end - CHUNK_SIZE_SECONDS + 1, CHUNK_SIZE_SECONDS):
        s, e = t * sfreq, (t + CHUNK_SIZE_SECONDS) * sfreq
        chunk = data[:, s:e]
        with np.errstate(divide='ignore', invalid='ignore'):
            norm = (chunk - np.mean(chunk, axis=1, keepdims=True)) / np.std(chunk, axis=1, keepdims=True)
            norm = np.nan_to_num(norm)
        chunks.append(norm.T)
    return chunks

def create_dataset_from_file(file_name):
    raw = load_edf(os.path.join(BASE_DATA_PATH, file_name))
    annotation_file = 'Seizures-list-' + file_name.split('-')[0] + '.txt'
    seizure_times = load_seizure_times(os.path.join(ANNOTATION_BASE_PATH, annotation_file))
    total_sec = int(raw.n_times / raw.info['sfreq'])
    preictal, interictal = [], []
    for sz in seizure_times:
        preictal.extend(segment_eeg(raw, max(0, sz - 1800), sz))
        if sz + 1200 < total_sec:
            interictal.extend(segment_eeg(raw, sz + 600, sz + 1200))
    return preictal, interictal

# Step 6: Load dataset
X, Y = [], []
edf_files = [f for f in os.listdir(BASE_DATA_PATH) if f.endswith('.edf')]
for f in edf_files:
    pre, inter = create_dataset_from_file(f)
    X.extend(pre)
    Y.extend([1]*len(pre))
    X.extend(inter)
    Y.extend([0]*len(inter))

X = np.array(X)
Y = np.array(Y)

# Step 7: Split data
X_trainval, X_test, Y_trainval, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_trainval, Y_trainval, test_size=0.2, stratify=Y_trainval, random_state=42)

# Step 8: Class weights
class_weights_values = compute_class_weight(class_weight='balanced', classes=np.unique(Y_train), y=Y_train)
class_weight_dict = dict(enumerate(class_weights_values))

# Step 9: Data generator
class EEGDataGenerator(Sequence):
    def __init__(self, x, y, batch_size=8):
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.indices = np.arange(len(x))
        np.random.shuffle(self.indices)

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_idx = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        return np.array([self.x[i] for i in batch_idx]), np.array([self.y[i] for i in batch_idx])

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

train_generator = EEGDataGenerator(X_train, Y_train)
val_generator = EEGDataGenerator(X_val, Y_val)
test_generator = EEGDataGenerator(X_test, Y_test)

# Step 10: Build model
timesteps, channels = X.shape[1], X.shape[2]

model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True), input_shape=(timesteps, channels)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
])

# Step 11: Train model
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    '/content/drive/MyDrive/BILSTM_MODEL/best_seizure_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    class_weight=class_weight_dict,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
        checkpoint_cb
    ]
)

# Step 12: Evaluate model and get predictions
results = model.evaluate(test_generator)
print(f"Test Accuracy: {results[1]:.4f}")
print(f"Test Precision: {results[2]:.4f}")
print(f"Test Recall: {results[3]:.4f}")

y_pred = (model.predict(test_generator) > 0.5).astype(int).flatten()
y_true = np.concatenate([y for _, y in test_generator])

# Step 13: Save model, predictions and history
model.save('/content/drive/MyDrive/BILSTM_MODEL/final_seizure_model.h5')

import pickle
with open('/content/drive/MyDrive/BILSTM_MODEL/history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

np.save('/content/drive/MyDrive/BILSTM_MODEL/y_true.npy', y_true)
np.save('/content/drive/MyDrive/BILSTM_MODEL/y_pred.npy', y_pred)

print("✅ Model, history, and predictions saved successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/30
556/556 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.5048 - loss: 0.7503 - precision: 0.7416 - recall: 0.4792
Epoch 1: val_loss improved from inf to 0.67345, saving model to /content/drive/MyDrive/BILSTM_MODEL/best_seizure_model.h5


556/556 ━━━━━━━━━━━━━━━━━━━━ 87s 143ms/step - accuracy: 0.5048 - loss: 0.7503 - precision: 0.7417 - recall: 0.4793 - val_accuracy: 0.6709 - val_loss: 0.6735 - val_precision: 0.7796 - val_recall: 0.7602
Epoch 2/30
556/556 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.6202 - loss: 0.6890 - precision: 0.8150 - recall: 0.6115
Epoch 2: val_loss improved from 0.67345 to 0.63994, saving model to /content/drive/MyDrive/BILSTM_MODEL/best_seizure_model.h5


556/556 ━━━━━━━━━━━━━━━━━━━━ 80s 143ms/step - accuracy: 0.6203 - loss: 0.6889 - precision: 0.8150 - recall: 0.6116 - val_accuracy: 0.6520 - val_loss: 0.6399 - val_precision: 0.8519 - val_recall: 0.6286
Epoch 3/30
556/556 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.6652 - loss: 0.6161 - precision: 0.8531 - recall: 0.6465
Epoch 3: val_loss improved from 0.63994 to 0.58753, saving model to /content/drive/MyDrive/BILSTM_MODEL/best_seizure_model.h5


556/556 ━━━━━━━━━━━━━━━━━━━━ 78s 140ms/step - accuracy: 0.6652 - loss: 0.6160 - precision: 0.8531 - recall: 0.6465 - val_accuracy: 0.7023 - val_loss: 0.5875 - val_precision: 0.8465 - val_recall: 0.7193
Epoch 4/30
556/556 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.7314 - loss: 0.5556 - precision: 0.8828 - recall: 0.7216
Epoch 4: val_loss improved from 0.58753 to 0.58590, saving model to /content/drive/MyDrive/BILSTM_MODEL/best_seizure_model.h5


556/556 ━━━━━━━━━━━━━━━━━━━━ 78s 140ms/step - accuracy: 0.7314 - loss: 0.5556 - precision: 0.8828 - recall: 0.7216 - val_accuracy: 0.7140 - val_loss: 0.5859 - val_precision: 0.8607 - val_recall: 0.7217
Epoch 5/30
556/556 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.7767 - loss: 0.4812 - precision: 0.9053 - recall: 0.7729
Epoch 5: val_loss improved from 0.58590 to 0.54565, saving model to /content/drive/MyDrive/BILSTM_MODEL/best_seizure_model.h5


...

556/556 ━━━━━━━━━━━━━━━━━━━━ 77s 138ms/step - accuracy: 0.8177 - loss: 0.4162 - precision: 0.9320 - recall: 0.8059 - val_accuracy: 0.7572 - val_loss: 0.5254 - val_precision: 0.9134 - val_recall: 0.7342
Epoch 8/30
556/556 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.8571 - loss: 0.3378 - precision: 0.9498 - recall: 0.8454
Epoch 8: val_loss improved from 0.52535 to 0.46350, saving model to /content/drive/MyDrive/BILSTM_MODEL/best_seizure_model.h5


556/556 ━━━━━━━━━━━━━━━━━━━━ 82s 139ms/step - accuracy: 0.8571 - loss: 0.3378 - precision: 0.9498 - recall: 0.8454 - val_accuracy: 0.8076 - val_loss: 0.4635 - val_precision: 0.8966 - val_recall: 0.8298
Epoch 9/30
556/556 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.9005 - loss: 0.2577 - precision: 0.9637 - recall: 0.8952
Epoch 9: val_loss improved from 0.46350 to 0.44914, saving model to /content/drive/MyDrive/BILSTM_MODEL/best_seizure_model.h5


556/556 ━━━━━━━━━━━━━━━━━━━━ 77s 139ms/step - accuracy: 0.9005 - loss: 0.2578 - precision: 0.9637 - recall: 0.8952 - val_accuracy: 0.8165 - val_loss: 0.4491 - val_precision: 0.9088 - val_recall: 0.8298
Epoch 10/30
556/556 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.8945 - loss: 0.2696 - precision: 0.9647 - recall: 0.8869
Epoch 10: val_loss improved from 0.44914 to 0.42269, saving model to /content/drive/MyDrive/BILSTM_MODEL/best_seizure_model.h5


556/556 ━━━━━━━━━━━━━━━━━━━━ 77s 138ms/step - accuracy: 0.8945 - loss: 0.2696 - precision: 0.9647 - recall: 0.8869 - val_accuracy: 0.8462 - val_loss: 0.4227 - val_precision: 0.9054 - val_recall: 0.8795
Epoch 11/30
556/556 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.9159 - loss: 0.2335 - precision: 0.9650 - recall: 0.9170
Epoch 11: val_loss improved from 0.42269 to 0.41137, saving model to /content/drive/MyDrive/BILSTM_MODEL/best_seizure_model.h5


556/556 ━━━━━━━━━━━━━━━━━━━━ 83s 139ms/step - accuracy: 0.9158 - loss: 0.2336 - precision: 0.9650 - recall: 0.9170 - val_accuracy: 0.8516 - val_loss: 0.4114 - val_precision: 0.8980 - val_recall: 0.8969
Epoch 12/30
556/556 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.9169 - loss: 0.2325 - precision: 0.9687 - recall: 0.9145
Epoch 12: val_loss did not improve from 0.41137
556/556 ━━━━━━━━━━━━━━━━━━━━ 76s 138ms/step - accuracy: 0.9169 - loss: 0.2325 - precision: 0.9687 - recall: 0.9145 - val_accuracy: 0.8138 - val_loss: 0.5070 - val_precision: 0.9153 - val_recall: 0.8186
Epoch 13/30
556/556 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.9293 - loss: 0.2030 - precision: 0.9736 - recall: 0.9268
Epoch 13: val_loss did not improve from 0.41137
556/556 ━━━━━━━━━━━━━━━━━━━━ 77s 139ms/step - accuracy: 0.9293 - loss: 0.2030 - precision: 0.9736 - recall: 0.9268 - val_accuracy: 0.8615 - val_loss: 0.4194 - val_precision: 0.8763 - val_recall: 0.9416
Epoch 14/30
556/556 ━━━━━━━━━━━━━━━━━━━━ 0s 12

In [ ]:
# ✅ Save training history to Drive
import pickle
with open('/content/drive/MyDrive/EEG_Siena_Seizure_Data/training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

print("✅ Training history saved!")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/EEG_Siena_Seizure_Data/training_history.pkl'